In [2]:
ls LibriSpeech/

BOOKS.TXT     LICENSE.TXT   SPEAKERS.TXT
CHAPTERS.TXT  README.TXT    dev-clean/


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os


In [8]:
chapters = pd.read_csv('LibriSpeech/CHAPTERS.TXT', delimiter='|')
chapters.head()

,ID,READER,MINUTES,SUBSET,PROJ.,BOOK ID,CH. TITLE,PROJECT TITLE
0,1,110,19.77,train-other-500,53,1023,In Chancery,Bleak House
1,2,110,10.30,train-other-500,53,1023,In Fashion,Bleak House
2,159,4174,7.67,train-other-500,68,2184,Letter XXV,Unbeaten Tracks in Japan
3,198,19,8.42,train-clean-100,219,121,Chapter 01,Northanger Abbey
4,199,98,11.68,train-clean-360,219,121,Chapter 02,Northanger Abbey


In [ ]:
## for dir in dev-clean:
## for folder in dir:
## folder-name + text = pd.read_csv(last file in directory)

In [49]:
import os

for filename in os.listdir('LibriSpeech/dev-clean/'):        
    try:
        for file in os.listdir('LibriSpeech/dev-clean/' + filename):
    
            text_file = glob.glob('LibriSpeech/dev-clean/' + filename + '/' + file + '/*txt')
            print(text_file[0])
#             print('LibriSpeech/dev-clean/' + filename + file + '/*')
#             latest_file = max(list_of_files, key=os.path.getctime)
            text_ref = pd.read_csv(text_file[0], header=[['ref', 'word1']], delimiter=' ')
            break
    except Exception:
        pass
        
#     if filename.endswith(".asm") or filename.endswith(".py"): 
#          # print(os.path.join(directory, filename))
#         continue
#     else:
#         continue

LibriSpeech/dev-clean/2412/153954/2412-153954.trans.txt
LibriSpeech/dev-clean/2078/142845/2078-142845.trans.txt
LibriSpeech/dev-clean/3752/4943/3752-4943.trans.txt
LibriSpeech/dev-clean/1988/24833/1988-24833.trans.txt
LibriSpeech/dev-clean/1919/142785/1919-142785.trans.txt
LibriSpeech/dev-clean/3000/15664/3000-15664.trans.txt
LibriSpeech/dev-clean/1993/147965/1993-147965.trans.txt
LibriSpeech/dev-clean/6313/76958/6313-76958.trans.txt
LibriSpeech/dev-clean/251/136532/251-136532.trans.txt
LibriSpeech/dev-clean/8842/302201/8842-302201.trans.txt
LibriSpeech/dev-clean/5536/43358/5536-43358.trans.txt
LibriSpeech/dev-clean/7850/286674/7850-286674.trans.txt
LibriSpeech/dev-clean/174/168635/174-168635.trans.txt
LibriSpeech/dev-clean/6295/64301/6295-64301.trans.txt
LibriSpeech/dev-clean/7976/110124/7976-110124.trans.txt
LibriSpeech/dev-clean/2803/154320/2803-154320.trans.txt
LibriSpeech/dev-clean/777/126732/777-126732.trans.txt
LibriSpeech/dev-clean/5694/64038/5694-64038.trans.txt
LibriSpeech/de

In [50]:
text_ref.head()

,2035-147961-0000,PETER,TOLD,HIS,TROUBLES,TO,MISTER,SHIMERDA,HE,WAS,...,ON.2,HIS.1,PIGS,AND.1,HORSES,AND.2,EVEN,HIS.2,MILK,COW
0,2035-147961-0001,PETER,COULD,GIVE,NO,VERY,CLEAR,ACCOUNT,OF,HIS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2035-147961-0002,SHE,ASKED,PETER,TO,WAIT,A,MOMENT,AND,WHEN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2035-147961-0003,WE,LAY,STILL,AND,DID,NOT,TALK,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2035-147961-0004,THE,LITTLE,HOUSE,ON,THE,HILLSIDE,WAS,SO,MUCH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2035-147961-0005,THEY,MADE,ME,THINK,OF,DEFEATED,ARMIES,RETREATING,OR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
import glob
import os

list_of_files = glob.glob('LibriSpeech/dev-clean/84/*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
print(latest_file)

LibriSpeech/dev-clean/84/121550
